In [1]:
import pandas as pd

card_infos = pd.read_csv('./hana_cardInfos.csv')

card_urls = card_infos['card_url'].tolist()
card_names = card_infos['card_name'].tolist()
card_imgs = card_infos['card_img'].tolist()
card_summarys = card_infos['card_summary'].tolist()

mID_urls = [str(url).zfill(5) for url in card_urls] # 다섯 자리로 맞추기

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

card_benefit = []

for i, url in enumerate(card_urls):
    url = f'https://www.hanacard.co.kr/OPI41000000D.web?schID=pcd&mID=PI410{mID_urls[i]}P&CD_PD_SEQ={url}&'
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    driver.implicitly_wait(20)
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    tab_list = soup.find_all('div', {'class': 'tab_cont'})
    benefit = ''

    if len(tab_list) == 0: # 탭이 하나도 없는 경우
        card_view_detail = soup.find('div', {'class': 'card_view_detail'})
        info_list = card_view_detail.find('ul', {'class': 'card_li'}).find_all('li', {'class': 'list'})
        
        for info in info_list:
            tit = info.find('dt', {'class': 'tit'})
            tit = str(tit.get_text(separator=' ', strip=True))
            benefit += f'[{tit}]'
                
            info_txt_list = info.find('dd', {'class': 'txt'}).find_all('li', {'class': 'blt1'})
            for txt in info_txt_list:
                info_txt = str(txt.get_text(separator=' ', strip=True))
                benefit += f'{info_txt} '
            
    elif len(tab_list) == 1:  
        title = tab_list[0].find('h5', {'class': 'blind'})
        if title:
            title = str(title.get_text(separator=' ', strip=True))
            benefit += f'###{title}'

        cont_list = tab_list[0].find_all('div', {'class': 'cont'})

        for cont in cont_list:
            cont_tit = cont.find('h6', {'class': 't_tit'})
            if cont_tit:
                cont_tit = str(cont_tit.get_text(separator=' ', strip=True))
                benefit += f'[{cont_tit}]'
                
            tables = cont.select('table')
            for table_num, table in enumerate(tables):

                # 표가 있을 때만 출력
                if table:
                    caption = table.find('caption').text
                    benefit += caption
                    rows = table.find_all('tr')
                    for j, row in enumerate(rows):
                        cells = row.find_all(['th', 'td'])
                        row_data = []

                        for cell in cells:
                            colspan = int(cell.get('colspan', 1))
                            content = cell.text.strip()
                            row_data.extend([content] * colspan)

                        row_string = ' '.join(['({})'.format(data) for data in row_data])
                            
                        # 각 행을 설명하는 문장 출력
                        sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                        benefit += sentence
                            
            cont_ul = cont.find_all('ul', recursive=False)
            if cont_ul:
                for ul in cont_ul:
                    cont_li = ul.find_all('li', recursive=False)
                    for li in cont_li:
                        # li 태그 하위에 table이 없는 경우에만 처리
                        if not li.find('table'):
                           benefit += li.text    
    else:
        # 주요 혜택
        title = tab_list[0].find('h5', {'class': 'blind'}).text
        benefit += f'###{title}'
        
        info_list = tab_list[0].find('ul', {'class': 'card_info_list'}).find_all('li')

        for info in info_list:
            tit = info.find('div', {'class': 'tit'}).find('p')
            tit = str(tit.get_text(separator=' ', strip=True))
            benefit += f'[{tit}]'
                
            info_txt = info.find('div', {'class': 'inner'}).find('p')
            info_txt = str(info_txt.get_text(separator=' ', strip=True))
            benefit += info_txt
                     
        # 주요 혜택 제외 나머지 탭
        for tab in tab_list[1:]:
            title = tab.find('h5', {'class': 'blind'})
            if title:
                title = str(title.get_text(separator=' ', strip=True))
                benefit += f'###{title}'

            cont_list = tab.find_all('div', {'class': 'cont'})

            for cont in cont_list:
                cont_tit = cont.find('h6', {'class': 't_tit'})
                if cont_tit:
                    cont_tit = str(cont_tit.get_text(separator=' ', strip=True))
                    benefit += f'[{cont_tit}]'

                
                tables = cont.select('table')
                for table_num, table in enumerate(tables):
                    # 표가 있을 때만 출력
                    if table:
                        caption = table.find('caption').text
                        benefit += caption
                        rows = table.find_all('tr')
                        for j, row in enumerate(rows):
                            cells = row.find_all(['th', 'td'])
                            row_data = []

                            for cell in cells:
                                colspan = int(cell.get('colspan', 1))
                                content = cell.text.strip()
                                row_data.extend([content] * colspan)

                            row_string = ' '.join(['({})'.format(data) for data in row_data])
                            
                            # 각 행을 설명하는 문장 출력
                            sentence = f"표의 {j + 1}번째 행은 {row_string}로 이루어져 있습니다."
                            benefit += sentence
                            
                cont_ul = cont.find_all('ul', recursive=False)
                if cont_ul:
                    for ul in cont_ul:
                        cont_li = ul.find_all('li', recursive=False)
                        for li in cont_li:
                            # li 태그 하위에 table이 없는 경우에만 처리
                            if not li.find('table'):
                                benefit += li.text
                            
    card_benefit.append(benefit)

driver.quit()

data = {"card_names": card_names, "card_imgs": card_imgs, "card_summarys": card_summarys, "card_benefits" : card_benefit}
df = pd.DataFrame(data)

df.to_csv("hana_benefits.csv", encoding = "utf-8-sig")

In [3]:
shinhan_benefits = pd.read_csv('./hana_benefits.csv')
shinhan_benefits

,Unnamed: 0,card_names,card_imgs,card_summarys,card_benefits
0,0,#MY WAY(샵 마이웨이) 카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,"1. 대중교통,커피,편의점 건당 300원 청구할인 2. 디지털 구독, 주유/충전 ...","###주요혜택[대중교통]버스, 지하철 건당 3백원 청구할인[커피]스타벅스, 커피빈,..."
1,1,#any 하나카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 디지털 구독 50% 청구할인 2. 디지털 결제 1% 청구할인 3. 국내외 전...,###주요혜택[디지털 이용 할인]디지털 구독 50% 청구할인 디지털 결제 1% 청구...
2,2,에너지 더블 카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 에너지 공과금 10% 청구할인 2. 일상생활 5% 청구할인 3. EV(전기차...,###주요혜택[에너지 공과금]10% 청구할인[일상생활]5% 청구할인[EV(전기차)충...
3,3,MULTI Living 모바일카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 온라인 쇼핑/마트 주중 최대 10% 할인 2. 주유/백화점 주말 최대 10%...,"###주요혜택[주중 할인]온라인쇼핑,마트 각각 월 최대 5%/10% 청구할인 (월 ..."
4,4,#tag1카드 Orange,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 커피 25~50% 할인 2. 편의점/통신요금/쇼핑 5~10% 할인 3. 대중...,"###주요혜택[커피할인]스타벅스, 커피빈, 카페베네, 커핀그루나루, 탐앤탐스, 파스..."
5,5,#tag1카드 Navy,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 학원/아파트관리비 5~10% 할인 2. 통신요금/쇼핑 5~10% 할인 3. ...,###주요혜택[교육 할인]학원업종 5~10% 할인[아파트관리비 할인]아파트관리비 자...
6,6,국민 행복 카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 보육료/유아학비 본인부담금 할인 1만원 청구할인 2. 커피 10% 청구할인 ...,"###주요혜택[보육료/유아학비 본인 부담금 할인]어린이집, 유치원 결제금액 중 본인..."
7,7,그린 카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,"1. 에코머니 포인트 적립 2. 문화, 레저시설 최대 50% 할인 3. 대형 할인...",###주요혜택[Point & Service]국내 전 가맹점 0.2~0.8% 에코머니...
8,8,하나 원큐 카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 하나원큐페이 결제 1.8% 적립 2. 국내 전가맹점 0.8% 적립 3. 해외...,###주요혜택[하나원큐페이 결제 적립]하나원큐페이에 카드를 등록하여 결제하는 건 1...
9,9,대구로 카드,https://www.hanacard.co.kr/ATTACH/NEW_HOMEPAGE...,1. 대구로 택시 앱 10% 청구할인 2. 대구로 앱 10% 청구할인 3. 생활밀...,###주요혜택[대구로 택시 앱](특화서비스)대구로 택시 앱 결제 시 10% 청구할인...
